In [67]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from tqdm.notebook import tqdm
import sys
import numpy as np
sys.path.append('..')
import config
import os
import csv
import time
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from multiprocessing import Process
from opencc import OpenCC

# 初始化转换器，t2s表示从繁体转简体
cc = OpenCC('t2s')

data_path_prefix = config.data_path
vector_db_path = config.vector_db_path
os.environ["OPENAI_API_KEY"] = config.api_key
os.environ["OPENAI_API_BASE"] = config.api_base

api_key_ali = config.api_key_ali
api_base_ali = config.api_base_ali

model_name = config.model
model_name = 'gpt-4'
retrive_top_k = config.top_k

In [22]:
embed_model = OpenAIEmbeddings(
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
)

In [68]:
def chat(system_message=None, human_message=None, model_name=model_name):
    answer = ''
    api_key = config.api_key
    api_base = config.api_base
    message=[{"role":"system","content":system_message},{"role":"user","content":human_message}]
    client = ChatOpenAI(
        openai_api_key=api_key,
        openai_api_base=api_base,
        model=model_name,
        temperature=0.2,
    )
    answer = client.invoke(message).content.strip()
    return answer

In [24]:
# 读取tag.txt文件
tags_data = pd.read_csv(os.path.join(data_path_prefix, 'tag.txt'), sep='\t', header=None, names=['user_id', 'tags'])
# 提取所有的标签，并统计每个标签的出现次数
tag_counts = dict()
all_tags = tags_data['tags'].dropna().str.split('_')
for tag in tqdm(all_tags):
    for t in tag:
        simplified_text = cc.convert(t)
        tag_counts[simplified_text] = tag_counts.get(simplified_text, 0) + 1

  0%|          | 0/275717 [00:00<?, ?it/s]

目前标签的处理把全部的标签分为3个部分
1. 重要标签：出现次数最多的前500个标签，处理流程如下
    - 使用GPT3.5逐个进行描述
    - 将标签及其描述使用text-embedding-ada-002模型进行嵌入，存入向量数据库
    - 将500个标签逐个视为query，从向量数据库中进行查询，获取top-2的结果，其中第一个为其本身，第二个为除自身外语义最相近的标签
    - 计算两个结果分数之间的差值，选取差值小于0.1的标签对，将两个标签进行合并，合并逻辑：
        - 首先考虑标签本身长度，希望重要标签能够尽可能地精简，处理如“偶稀饭睡觉觉”和“睡觉”这样类型的标签
        - 其次，当长度一致时，考虑标签出现次数，出现次数较多的将会覆盖较少的，处理如“旅游”和“旅行”这样类型的标签
    - 最终处理结束后，有304个重要标签
2. 长尾标签：出现次数大于等于10次的标签（9400），处理流程如下
    - 使用Qwen-max逐个进行描述
    - 将标签及其描述使用text-embedding-ada-002模型进行嵌入
    - 使用嵌入向量从向量数据库中进行检索，检索top-10结果，作为候选集
    - 将标签及其描述与候选集一起输入给Qwen-max，让其从候选集中选择一个与输入标签最相关的标签
    - 如果Qwen-max判断候选集中不存在相关标签，则返回None，此时将该标签保留，不进行替换
3. 罕见标签：出现次数小于10次的标签（27w），处理流程如下
    - 将标签直接进行嵌入，使用嵌入向量进行检索，检索top-1的结果，进行替换

In [28]:
# 将标签出现次数转化为数据框
tag_count_distribution = pd.DataFrame.from_dict(tag_counts, orient='index', columns=['count']).sort_values('count', ascending=False)

# 重要标签
k = 500
main_tags = tag_count_distribution[:k]
# 长尾标签
long_tail_tags = tag_count_distribution[((tag_count_distribution['count']>=10) & (tag_count_distribution['count']<=20)) | (tag_count_distribution['count']>=26)][k:]
# 罕见标签
rare_tags = tag_count_distribution[tag_count_distribution['count']<10]

In [74]:
def vectorstore_from_kg(data=None, output_path: str = None, embed_model=embed_model, save=True
):
    if data is None:
        data = []
    if not os.path.exists(output_path):
        feature = data
        vectorstore = FAISS.from_texts(texts=feature, embedding=embed_model)
        if save:
            vectorstore.save_local(output_path)
    else:
        vectorstore = FAISS.load_local(output_path, embed_model, allow_dangerous_deserialization=True)
    return vectorstore

In [75]:
data_list = []
for i in range(10):
    data = pd.read_csv(os.path.join(data_path_prefix, 'rare_tags_{}.csv'.format(i)), header=None)
    data_list.append(data)
data_fin = pd.concat(data_list, ignore_index=True)
data_fin.to_csv(os.path.join(data_path_prefix, 'rare_tags.csv'), index=False, header=False)

In [76]:
# 对重要标签进行处理，对齐其中语义相近的标签
tag_des = []
split_flag = '，该标签的描述为：'
if os.path.exists(os.path.join(data_path_prefix, "main_tags.csv")):
    df = pd.read_csv(os.path.join(data_path_prefix, "main_tags.csv"))
    # 读取向量数据库
    vector_db = vectorstore_from_kg(output_path=vector_db_path)
else:
    # 将标签逐个进行描述，增加额外的信息，使得嵌入向量更加合理
    for tag in tqdm(main_tags.index):
        system_messgae = "你是一个对用户兴趣标签十分了解的专家，请在50个字以内描述一下输入给你的用户兴趣标签。请注意，输入的词语并不是单纯的词，而是一个用户的兴趣，你需要做的是阐述当这个词被当作用户画像的一部分时，它的含义是什么。"
        human_message = tag
        answer = chat(system_messgae, human_message)
        tag_des.append(answer)
    main_tags["tag_des"] = tag_des

    # 把标签及其描述处理成如[音乐，该标签的描述为XXX]的形式
    data = list(
        main_tags.reset_index(drop=False).apply(
            lambda x: x[0] + split_flag + x["tag_des"], axis=1
        )
    )
    # 创建临时向量数据库
    vector_db_temp = vectorstore_from_kg(data, vector_db_path, save=False)

    # 对重要标签中的每个标签，都从向量数据库中检索top2的标签，其中第一个为该标签本身，第二个为与其相似的标签
    data_list = []
    for i in tqdm(data):
        retrieve = [
            (j.page_content, s)
            for j, s in vector_db_temp.similarity_search_with_relevance_scores(i, 2)
        ]
        data = [
            retrieve[0][0],  # tag1
            tag_count_distribution.loc[retrieve[0][0].split(split_flag)[0]][
                "count"
            ],  # tag1_count
            retrieve[1][0],  # tag2
            tag_count_distribution.loc[retrieve[1][0].split(split_flag)[0]][
                "count"
            ],  # tag2_count
            retrieve[0][1]
            - retrieve[1][1],  # tag1与tag2检索分数的差值，用于衡量两个标签有多相近
        ]
        data_list.append(data)
    df = pd.DataFrame(
        data_list,
        columns=["tag1", "tag1_count", "tag2", "tag2_count", "relevance_score_diff"],
    )

    # 将相近的标签进行对齐，生成最终的tag
    df["final_tag"] = df.apply(
        lambda x: (
            x["tag1"]
            if x["relevance_score_diff"]
            >= 0.1  # 差值大于0.1的不考虑合并，经观察，没有合并的必要
            or (
                len(x["tag1"].split(split_flag)[0])
                == len(x["tag2"].split(split_flag)[0])
                and x["tag1_count"] >= x["tag2_count"]
            )
            or len(x["tag1"].split(split_flag)[0])
            < len(
                x["tag2"].split(split_flag)[0]
            )  # 考虑标签的长度，希望最终的标签尽可能短，过长的标签有较大可能是如“偶稀饭睡觉觉”这类描述性标签，转化为“爱睡觉”是比较合理的。另外，当标签长度一致的时候，使用出现次数进行进一步的判断。
            else x["tag2"]
        ),
        axis=1,
    )
    
    # 把标签及其描述处理成如[音乐，该标签的描述为XXX]的形式
    data = list(df['final_tag'].drop_duplicates())
    # 创建向量数据库
    vector_db = vectorstore_from_kg(data, vector_db_path, save=True)
    
    df.to_csv(os.path.join(data_path_prefix, "main_tags.csv"), index=False)

In [71]:
# 对长尾标签进行处理
if os.path.exists(os.path.join(data_path_prefix, 'processed_tags.csv')):
    data_temp = pd.read_csv(os.path.join(data_path_prefix, 'processed_tags.csv'), header=None)
    check_point = data_temp[0].max()
else:
    check_point = 0
long_tail_tags_list= long_tail_tags.index.tolist()[check_point:]
error_list = []
for i, tag in tqdm(enumerate(long_tail_tags_list), total=len(long_tail_tags_list)):
    try:
        # 为每个tag生成一段描述
        system_messgae = "你是一个对用户兴趣标签十分了解的专家，你的输入是一个用户的兴趣标签，请在10个字以内描述一下该标签。"
        human_message = tag
        answer = chat(system_messgae, human_message)
        tag_with_desc = tag + split_flag + answer
        # 利用标签及其描述进行检索
        retrieve_tags = [(i.page_content.split(split_flag)[0]) for i, _ in vector_db.similarity_search_with_relevance_scores(tag_with_desc, retrive_top_k)]
        
        # 读取分组的prompt
        with open('./prompt.txt', 'r') as f:
            prompt = f.read()
        # 进行标签分组
        system_messgae = '你是一个标签分组的标注员，请协助用户进行多标签的分组工作'
        human_message = prompt.replace('<<<tag>>>', tag_with_desc).replace('<<<candidates>>>', ','.join(retrieve_tags))
        
        answer = chat(system_messgae, human_message)
        data_for_tag = [i+check_point, tag, answer, retrieve_tags, tag_with_desc]
        # 将结果持续追加至本地
        with open(os.path.join(data_path_prefix, 'processed_tags.csv'), 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(data_for_tag)
        time.sleep(2)
    except Exception as e:
        print(i+check_point, tag, e)
        error_list.append((i+check_point, tag, e))

  0%|          | 0/7200 [00:00<?, ?it/s]

1402 神 Error code: 400 - {'error': {'code': 'ResponseTimeout', 'param': None, 'message': 'Response timeout!', 'type': 'ResponseTimeout'}}
1570 工艺品 Error code: 400 - {'error': {'code': 'ResponseTimeout', 'param': None, 'message': 'Response timeout!', 'type': 'ResponseTimeout'}}
1595 宅腐 Error code: 400 - {'error': {'code': 'ResponseTimeout', 'param': None, 'message': 'Response timeout!', 'type': 'ResponseTimeout'}}
1814 强迫症偏执狂 Error code: 400 - {'error': {'code': 'Arrearage', 'param': None, 'message': 'Access denied, please make sure your account is in good standing.', 'type': 'Arrearage'}}
1815 80后宅男 Error code: 400 - {'error': {'code': 'Arrearage', 'param': None, 'message': 'Access denied, please make sure your account is in good standing.', 'type': 'Arrearage'}}
1816 诺基亚 Error code: 400 - {'error': {'code': 'Arrearage', 'param': None, 'message': 'Access denied, please make sure your account is in good standing.', 'type': 'Arrearage'}}
1817 育儿知识 Error code: 400 - {'error': {'code': 'Ar

In [72]:
with open(os.path.join(data_path_prefix, 'error_list.csv'), 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(error_list)

In [42]:
# 对罕见标签进行处理
# 将罕见标签分为10份，每份进行处理
point = []
for i in range(10):
    point.append((i+1) * len(rare_tags)//10)
point.pop()
point.append(len(rare_tags))

# 定义处理函数
def rare_tag_process(start=0, end=1, process=0):
    rare_tags_list = rare_tags.index.tolist()[start:end]
    for i, tag in tqdm(enumerate(rare_tags_list), total=len(rare_tags_list)):
        result = vector_db.similarity_search_with_relevance_scores(tag, 1)[0][0].page_content.split(split_flag)[0]
        data = [i+start, tag, result]
        with open(os.path.join(data_path_prefix, f'rare_tags_{process}.csv'), 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(data)

# 多进程处理
process_list = []
for i in range(10):
    if i == 0:
        start = 0
    else:
        start = point[i-1]
    end = point[i]
    process = Process(target=rare_tag_process, args=(start, end, i))
    process_list.append(process)

for i in range(10):
    process_list[i].start()

for i in range(10):
    process_list[i].join()

In [129]:
# 长尾标签后处理
# tag1 = pd.read_csv('/mnt/social_network_course/data/weibo_user_data/processed_tags_21_26.csv', header=None)
# tag2 = pd.read_csv('/mnt/social_network_course/data/weibo_user_data/processed_tags.csv', header=None)
# tag_processed = pd.concat([tag1, tag2])[[1, 2]].rename(columns={1: 'tag', 2: 'final_tag'})
# tag_total = long_tail_tags.reset_index(drop=False)
# tag_join = tag_total.merge(tag_processed, left_on='index', right_on='tag', how='left')
# tag_na = tag_join[(tag_join['final_tag'].isna()) & (~tag_join['tag'].isna())]
# tag_na['final_tag'] = tag_na['tag']
# temp = []
# for tag in tag_join[tag_join['tag'].isna()]['index']:
#     # 为每个tag生成一段描述
#     system_messgae = "你是一个对用户兴趣标签十分了解的专家，你的输入是一个用户的兴趣标签，请在10个字以内描述一下该标签。"
#     human_message = tag
#     answer = chat(system_messgae, human_message)
#     tag_with_desc = tag + split_flag + answer
#     # 利用标签及其描述进行检索
#     retrieve_tags = [(i.page_content.split(split_flag)[0]) for i, _ in vector_db.similarity_search_with_relevance_scores(tag_with_desc, retrive_top_k)]
    
#     # 读取分组的prompt
#     with open('./prompt.txt', 'r') as f:
#         prompt = f.read()
#     # 进行标签分组
#     system_messgae = '你是一个标签分组的标注员，请协助用户进行多标签的分组工作'
#     human_message = prompt.replace('<<<tag>>>', tag_with_desc).replace('<<<candidates>>>', ','.join(retrieve_tags))
    
#     answer = chat(system_messgae, human_message)
#     temp.append([tag, answer, retrieve_tags, tag_with_desc])
# tag_error = {}
# for i in temp:
#     tag_error[i[0]] = i[1]
# tag_error = pd.DataFrame(tag_error.items(), columns=['tag', 'final_tag'])
# tag_join = tag_join.dropna()[['tag', 'final_tag']]
# tag_na = tag_na[['tag', 'final_tag']]
# tag_long_tail = pd.concat([tag_join, tag_na, tag_error]).drop_duplicates('tag')
# tag_long_tail.to_csv('/mnt/social_network_course/data/weibo_user_data/long_tail_tags.csv')

/tmp/ipykernel_94689/3565208586.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tag_na['final_tag'] = tag_na['tag']


In [130]:
tag_long_tail['final_tag'] = tag_long_tail['final_tag'].apply(lambda x: x.split('choise: ')[-1])
tag_long_tail['final_tag'] = tag_long_tail['final_tag'].apply(lambda x: x.split(',')[0].replace('[', '').replace("'", ''))

In [137]:
temp_dict = {}
for i in tag_long_tail.to_dict(orient='records'):
    if i['final_tag'] not in main_tags.index:
        if i['final_tag'] != i['tag']:
            temp_dict[i['tag']] = i['final_tag']
tag_long_tail['final_tag'] = tag_long_tail['final_tag'].apply(lambda x: temp_dict['tag'] if x in temp_dict.keys() else x)

In [149]:
tag_long_tail['final_tag'] = tag_long_tail.apply(lambda x: x['tag'] if x['tag'] in temp_dict.keys() else x['final_tag'], axis=1)

In [150]:
tag_long_tail['str_length'] = tag_long_tail['final_tag'].apply(lambda x: len(x))
test = tag_long_tail.sort_values('str_length', ascending=False)[:300]
test

,tag,final_tag,str_length,test
3050,WindowsPhone,WindowsPhone,12,NaN
2917,gossipgirl,gossipgirl,10,NaN
8290,963693628,963693628,9,NaN
7380,粉红控KITTY控,粉红控KITTY控,9,NaN
4234,iphone爱好者,iphone爱好者,9,NaN
...,...,...,...,...
7780,团购网站大全,网购达人,4,NaN
4328,你懂的,幽默搞笑,4,NaN
4331,影楼,婚纱摄影,4,NaN
1132,婚纱控,婚纱摄影,4,NaN
